In [1]:
import pandas as pd
import world_trade_data as wits
pd.set_option('display.max_rows', 6)

In [43]:
pd.read_xml('http://wits.worldbank.org/API/V1/SDMX/V21/rest/dataflow/wbg_wits/')

,ID,Test,Prepared,Sender,Receiver,Dataflows
0,IDREF24,false,2023-02-09T14:23:03.2621444-05:00,NaN,NaN,NaN
1,None,None,None,NaN,NaN,NaN


In [41]:
import typing
import pandas as pd
import urllib.request
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import os 
import requests

In [44]:
requests.get('http://wits.worldbank.org/API/V1/SDMX/V21/rest/dataflow/wbg_wits/')

<Response [200]>

In [26]:
def get_feed(url):
    """Scrapes an XML feed from the provided URL and returns XML source.
    
    Args:
        url (string): Fully qualified URL pointing to XML feed.

    Returns:
        source (string): XML source of scraped feed.
    """
    
    try:
        response = urllib.request.urlopen(urllib.request.Request(url, headers={'User-Agent': 'Mozilla'}))
        source = BeautifulSoup(response, 'lxml-xml', from_encoding=response.info().get_param('charset'))
        return source
    except Exception as e:
        print('Error: ' + str(e))

In [29]:
FEED_URL = 'http://wits.worldbank.org/API/V1/wits/datasource/trn/product/all'
xml = get_feed(FEED_URL)

In [30]:
print(xml)

<?xml version="1.0" encoding="utf-8"?>
<wits:datasource datasourcecode="TRN" datasourcename="WITS - UNTCTAD TRAINS" language="en" total="6538" xmlns:wits="http://wits.worldbank.org">
<wits:products>
<wits:product grouptype="N/A" isgroup="No" nomenclaturecode="HS" productcode="010110">
<wits:productdescription>010110 -- (2002-2011) - Pure-bred breeding animals</wits:productdescription>
<wits:notes/>
</wits:product>
<wits:product grouptype="N/A" isgroup="No" nomenclaturecode="HS" productcode="010111">
<wits:productdescription>010111 -- (-2001) -- Pure-bred breeding animals</wits:productdescription>
<wits:notes/>
</wits:product>
<wits:product grouptype="N/A" isgroup="No" nomenclaturecode="HS" productcode="010119">
<wits:productdescription>010119 -- (-2001) -- Other</wits:productdescription>
<wits:notes/>
</wits:product>
<wits:product grouptype="N/A" isgroup="No" nomenclaturecode="HS" productcode="010120">
<wits:productdescription>010120 -- (-2001) - Asses, mules and hinnies</wits:productd

In [31]:
def get_elements(xml, item='item'):
    
    try:
        items = xml.find_all(item)
        elements = [element.name for element in items[0].find_all()]
        return elements
    except Exception as e:
        print('Error: ' + str(e))

In [33]:
def feed_to_df(url, item='item'):
    
    xml = get_feed(url)
    elements = get_elements(xml)
    
    if isinstance(elements, typing.List):
        df = pd.DataFrame(columns=elements)
    
        items = xml.find_all(item)
        
        for item in items:
            row = {}
            for element in elements:
                if xml.find(element):
                    if item.find(element):
                        row[element] = item.findNext(element).text
                    else:
                        row[element] = ''
                else:
                    row[element] = ''
            
            df = df.append(row, ignore_index=True)
        return df

In [34]:
df = feed_to_df(FEED_URL)

Error: list index out of range
